# fMRI Data Loading and Normalization in Python 
**V.0.2 - Beta, [Contributions](#contributions)**   

### Goal of this script
 1. load the fMRI data into python
     - 1 postfaces run
     - 2 familiarization runs
     - 4 reward runs
     - 2 decision runs
     - note that postscenes is part of ses-02, but was processed with the ses-01 runs
 2. create an average brain mask from multiple runs
     - ses02_brain (10 task runs)
 3. trim TRs from the beginning AND end of each run (and apply this trimming to the confounds as well). Number of volumes trimmed defined by n_trunc_beginning and n_trunc_end. 
     - save volume as _trimXandXTRs.nii.gz
 4. apply a high-pass filter and z-score the data
     - save volume as _trimXandXTRs_normalized.nii.gz
 5. concatenate runs to make one time series for various analyses
     - familiarization runs (2)
     - reward runs (4)
 6. plot a timeseries for a voxel

## Define subject

In [ ]:
sub = 'sub-XXX'
ses = 'ses-02'
task1='postFaces'
task2='familiarization'
task3='reward'
task4='decision'
n_trunc_beginning=9 #Number of volumes to trim from beginning of run
n_trunc_end=5 #Number of volumes to trim from end of run

## Import necessary packages

In [ ]:
import warnings
import sys 
if not sys.warnoptions:
    warnings.simplefilter("ignore")
import numpy as np
import pandas as pd
import nibabel as nib
from nilearn.input_data import NiftiMasker,  MultiNiftiMasker
from nilearn.masking import intersect_masks
from nilearn import image
from nilearn.plotting import plot_roi
from nilearn.plotting import plot_anat
from nilearn.plotting import plot_epi
from nilearn.image.image import mean_img
from scipy import stats
from sklearn import preprocessing
import matplotlib.pyplot as plt 
#import seaborn as sns 
import scipy.io

%matplotlib inline 
%autosave 5
#sns.set(style = 'white', context='poster', rc={"lines.linewidth": 2.5})

## Load settings

In [ ]:
# load some helper functions
sys.path.insert(0, '/jukebox/norman/emcdevitt/studies/SVD/code/mainanalysis')
import svd_utils
from svd_utils import load_svd_epi_data, load_data

# load some constants
from svd_utils import svd_dir, svd_bids_dir, svd_TR, svd_hrf_lag, run_names, n_runs, TRs_run

print('TASKS:', task1, task2, task3, task4)
print('LIST OF ALL TASKS:', run_names)
task1_index = run_names.index(task1)
task2_index = run_names.index(task2)
task3_index = run_names.index(task3)
task4_index = run_names.index(task4)
print('task index:', task1_index)
print('task index:', task2_index)
print('task index:', task3_index)
print('task index:', task4_index)
print('')

n_runs_postfaces = n_runs[task1_index]
n_runs_familiarization = n_runs[task2_index]
n_runs_reward = n_runs[task3_index]
n_runs_decision = n_runs[task4_index]
n_runs_total = n_runs_postfaces + n_runs_familiarization + n_runs_reward + n_runs_decision

TRs_run_postfaces=TRs_run[task1_index]
TRs_run_familiarization=TRs_run[task2_index]
TRs_run_reward=TRs_run[task3_index]
TRs_run_decision=TRs_run[task4_index]

anat_dir=svd_bids_dir + 'derivatives/deface/'
out_dir= svd_bids_dir + 'derivatives/firstlevel/%s/' % sub
mask_fold = svd_bids_dir + 'derivatives/firstlevel/%s/masks/' % sub

ses0_dir=svd_bids_dir + 'derivatives/fmriprep/%s/ses-00/func/' % sub
ses1_dir=svd_bids_dir + 'derivatives/fmriprep/%s/ses-01/func/' % sub
ses2_dir=svd_bids_dir + 'derivatives/fmriprep/%s/ses-02/func/' % sub

print('bids dir = %s' % (svd_bids_dir))
print('')
print('subject dir = %s' % (ses2_dir))
print('')
print('output dir = %s' % (out_dir))
print('')
print('number of postfaces runs = %d' % (n_runs_postfaces))
print('number of familiarization runs = %d' % (n_runs_familiarization))
print('number of reward runs = %d' % (n_runs_reward))
print('number of decision runs = %d' % (n_runs_decision))
print('TR = %s seconds' % (svd_TR))
print('TRs per postfaces run = %s' % (TRs_run_postfaces))
print('TRs per familiarization run = %s' % (TRs_run_familiarization))
print('TRs per reward run = %s' % (TRs_run_reward))
print('TRs per decision run = %s' % (TRs_run_decision))
print('trim %d volumes from beginning of each run' % (n_trunc_beginning))
print('trim %d volumes from end of each run' % (n_trunc_end))

## Select confounds and trim volumes from confounds file
Choose the desired confounds from the confounds_regressors.tsv file from fmriprep, trim the columns corresponding to trimmed volumes, and save as a .txt file. 

In [ ]:
# POSTFACES
task1='postfaces' #update for file naming purposes
confounds=[]
mc_all=[]
print(run_names[task1_index], 'Number of runs:', n_runs_postfaces, 'TRs per run:', TRs_run_postfaces, '-->', TRs_run_postfaces-n_trunc_beginning-n_trunc_end)

for r in range(1,n_runs_postfaces+1):
    fname='_ses-02_task-%s_run-0%i_desc-confounds_regressors.tsv' % (task1, r)
    confounds = pd.read_csv(ses2_dir + sub + fname,  sep='\t', header=(0))
    #confounds_selected=confounds[['trans_x','trans_y','trans_z','rot_x','rot_y','rot_z']][n_trunc_beginning:] #only trim beginning
    confounds_selected=confounds[['trans_x','trans_y','trans_z','rot_x','rot_y','rot_z']][n_trunc_beginning:-n_trunc_end] #trim beginning and end
    confounds_selected=pd.DataFrame(confounds_selected)
    confounds_selected.to_csv(out_dir + 'ses-02/' + sub + '_ses-02_task-%s_run-0%i_confounds_selected_trim%dand%dTRs.txt' % (task1, r, n_trunc_beginning, n_trunc_end), index=False, sep='\t', mode='w')
    print(confounds_selected.shape)
    
# FAMILIARIZATION
confounds=[]
mc_all=[]
print(run_names[task2_index], 'Number of runs:', n_runs_familiarization, 'TRs per run:', TRs_run_familiarization, '-->', TRs_run_familiarization-n_trunc_beginning-n_trunc_end)

for r in range(1,n_runs_familiarization+1):
    fname='_ses-02_task-%s_run-0%i_desc-confounds_regressors.tsv' % (task2, r)
    confounds = pd.read_csv(ses2_dir + sub + fname,  sep='\t', header=(0))
    #confounds_selected=confounds[['trans_x','trans_y','trans_z','rot_x','rot_y','rot_z']][n_trunc_beginning:] #only trim beginning
    confounds_selected=confounds[['trans_x','trans_y','trans_z','rot_x','rot_y','rot_z']][n_trunc_beginning:-n_trunc_end] #trim beginning and end
    confounds_selected=pd.DataFrame(confounds_selected)
    confounds_selected.to_csv(out_dir + 'ses-02/' + sub + '_ses-02_task-%s_run-0%i_confounds_selected_trim%dand%dTRs.txt' % (task2, r, n_trunc_beginning, n_trunc_end), index=False, sep='\t', mode='w')
    print(confounds_selected.shape)
    
# REWARD
confounds=[]
mc_all=[]
print(run_names[task3_index], 'Number of runs:', n_runs_reward, 'TRs per run:', TRs_run_reward, '-->', TRs_run_reward-n_trunc_beginning-n_trunc_end)

for r in range(1,n_runs_reward+1):
    fname='_ses-02_task-%s_run-0%i_desc-confounds_regressors.tsv' % (task3, r)
    confounds = pd.read_csv(ses2_dir + sub + fname,  sep='\t', header=(0))
    #confounds_selected=confounds[['trans_x','trans_y','trans_z','rot_x','rot_y','rot_z']][n_trunc_beginning:] #only trim beginning
    confounds_selected=confounds[['trans_x','trans_y','trans_z','rot_x','rot_y','rot_z']][n_trunc_beginning:-n_trunc_end] #trim beginning and end
    confounds_selected=pd.DataFrame(confounds_selected)
    confounds_selected.to_csv(out_dir + 'ses-02/' + sub + '_ses-02_task-%s_run-0%i_confounds_selected_trim%dand%dTRs.txt' % (task3, r, n_trunc_beginning, n_trunc_end), index=False, sep='\t', mode='w')
    print(confounds_selected.shape)
    
# DECISION
confounds=[]
mc_all=[]
print(run_names[task4_index], 'Number of runs:', n_runs_decision, 'TRs per run:', TRs_run_decision, '-->', TRs_run_decision-n_trunc_beginning-n_trunc_end)

for r in range(1,n_runs_decision+1):
    fname='_ses-02_task-%s_run-0%i_desc-confounds_regressors.tsv' % (task4, r)
    confounds = pd.read_csv(ses2_dir + sub + fname,  sep='\t', header=(0))
    #confounds_selected=confounds[['trans_x','trans_y','trans_z','rot_x','rot_y','rot_z']][n_trunc_beginning:] #only trim beginning
    confounds_selected=confounds[['trans_x','trans_y','trans_z','rot_x','rot_y','rot_z']][n_trunc_beginning:-n_trunc_end] #trim beginning and end
    confounds_selected=pd.DataFrame(confounds_selected)
    confounds_selected.to_csv(out_dir + 'ses-02/' + sub + '_ses-02_task-%s_run-0%i_confounds_selected_trim%dand%dTRs.txt' % (task4, r, n_trunc_beginning, n_trunc_end), index=False, sep='\t', mode='w')
    print(confounds_selected.shape)
    
print ('Confound files completed')

## Create an average mask

Make an average mask by intersecting the mask for each run (include postscenes run in ses-02 average brain mask). Plot average mask overlayed on subject's defaced T1. 

In [ ]:
mask_imgs=[]
for run in range(1,n_runs_total+2):
    if run==5:
        real_run=1
        mask_name = ses2_dir + sub + '_ses-02_task-postscenes_run-0%i_space-T1w_desc-brain_mask.nii.gz' % real_run
    elif run==6:
        real_run=1
        mask_name = ses2_dir + sub + '_ses-02_task-postfaces_run-0%i_space-T1w_desc-brain_mask.nii.gz' % real_run
    elif run==7:
        real_run=1
        mask_name = ses2_dir + sub + '_ses-02_task-familiarization_run-0%i_space-T1w_desc-brain_mask.nii.gz' % real_run
    elif run==8:
        real_run=2
        mask_name = ses2_dir + sub + '_ses-02_task-familiarization_run-0%i_space-T1w_desc-brain_mask.nii.gz' % real_run
    elif run==9:
        real_run=1
        mask_name = ses2_dir + sub + '_ses-02_task-decision_run-0%i_space-T1w_desc-brain_mask.nii.gz' % real_run
    elif run==10:
        real_run=2
        mask_name = ses2_dir + sub + '_ses-02_task-decision_run-0%i_space-T1w_desc-brain_mask.nii.gz' % real_run
    else:
        mask_name = ses2_dir + sub + '_ses-02_task-reward_run-0%i_space-T1w_desc-brain_mask.nii.gz' % run

    mask_imgs.append(mask_name)

# intersect brain masks
avg_mask=intersect_masks(mask_imgs, threshold=0.5, connected=True)

# plot
t1_file = anat_dir + sub + '_desc-preproc_T1w_defaced.nii.gz'
t1_img = image.load_img(t1_file)
plot_roi(avg_mask, bg_img=t1_img)

In [ ]:
# Save the mask
output_name = mask_fold + '%s_%s_brain.nii.gz' % (sub, ses)
print('Save average mask:', output_name)
print('')

dimsize=avg_mask.header.get_zooms()
affine_mat = avg_mask.affine
print('Mask dimensions:', dimsize)
print('')
print('Affine:')
print(affine_mat)

hdr = avg_mask.header  # get a handle for the .nii file's header
hdr.set_zooms((dimsize[0], dimsize[1], dimsize[2]))
nib.save(avg_mask, output_name)

## Drop first few TRs; apply mask and normalize data

`n_trunc_beginning` and `n_trunc_end` sets the number of TRs to drop from beginning and end of volume, respectively.

#### Get voxels from an ROI

We will extract BOLD data, only for voxels in a mask, by executing the following sequence of steps: 
1. load whole brain fMRI data (for a given subject and a given run)
2. trim TRs from beginning and end if desired
2. load the desired mask
3. use `NiftiMasker` to sub-select mask voxels from the whole brain data, apply a high-pass filter and normalize
    - `NiftiMasker` is a function from nilearn. Here's <a href="https://nilearn.github.io/auto_examples/04_manipulating_images/plot_mask_computation.html">an example</a> about how to use it, and here's the official <a href="https://nilearn.github.io/modules/generated/nilearn.input_data.NiftiMasker.html">documentation</a>.

In [ ]:
epi_mask_data_familiarization=[]
epi_mask_data_reward=[]
epi_mask_data_decision=[]

for run in range(1,n_runs_total+1):
    print('now on run:', run)
    epi_trunc=[]
    
    # define NiftiMasker function
    epi_masker= NiftiMasker(
        mask_img=avg_mask,  
        high_pass=1/128,
        standardize=True,  # Are you going to zscore the data across time?
        t_r=svd_TR, 
        memory='nilearn_cache',  # Caches the mask in the directory given as a string here so that it is easier to load and retrieve
        memory_level=1,  # How much memory will you cache?
        verbose=0
    )
    
    # load BOLD data
    if run==5:
        real_run=1
        epi_file = ses2_dir + sub + '_ses-02_task-postfaces_run-0%i_space-T1w_desc-preproc_bold.nii.gz' % real_run
    elif run==6:
        real_run=1
        epi_file = ses2_dir + sub + '_ses-02_task-familiarization_run-0%i_space-T1w_desc-preproc_bold.nii.gz' % real_run
    elif run==7:
        real_run=2
        epi_file = ses2_dir + sub + '_ses-02_task-familiarization_run-0%i_space-T1w_desc-preproc_bold.nii.gz' % real_run
    elif run==8:
        real_run=1
        epi_file = ses2_dir + sub + '_ses-02_task-decision_run-0%i_space-T1w_desc-preproc_bold.nii.gz' % real_run
    elif run==9:
        real_run=2
        epi_file = ses2_dir + sub + '_ses-02_task-decision_run-0%i_space-T1w_desc-preproc_bold.nii.gz' % real_run
    else:
        epi_file = ses2_dir + sub + '_ses-02_task-reward_run-0%i_space-T1w_desc-preproc_bold.nii.gz' % run

    epi_data=nib.load(epi_file)
    epi=epi_data.get_fdata()
    
    # trim beginning and end; call this epi_trunc
    epi_trunc =np.zeros((epi_data.shape[0], epi_data.shape[1], epi_data.shape[2], epi_data.shape[3]-n_trunc_beginning-n_trunc_end))
    epi_trunc[:, :, :, :] = epi[:,:,:,n_trunc_beginning:-n_trunc_end]
    
    # epi_trunc
    print('input filename: %s' % (epi_file))
    print('')
    print('Original:', epi_data.shape, '  ', 'Truncated:', epi_trunc.shape)
    dimsize=epi_data.header.get_zooms()
    print('Dimensions:', dimsize)
    orig_dimsize=dimsize
    affine_mat = epi_data.affine  #What is the orientation of the data?
    print('Affine:')
    print(affine_mat)
    print('')
    
    epi_trunc_nii = nib.Nifti1Image(epi_trunc, affine_mat) #convert to nifti
    
    # If we were saving the data from this intermediate step (trimmed but not normalized)
    #hdr = epi_trunc_nii.header  # get a handle for the .nii file's header
    #hdr.set_zooms((dimsize[0], dimsize[1], dimsize[2], dimsize[3]))
    #nib.save(epi_trunc_nii, output_name)
    
    # apply epi_masker function to truncated data
    epi_mask_data = epi_masker.fit_transform(epi_trunc_nii)
    
    # make concatenated data structures
    if run==1:
        epi_mask_data_reward=epi_mask_data
    elif run==2:
        epi_mask_data_reward=np.vstack([epi_mask_data_reward,epi_mask_data])
    elif run==3:
        epi_mask_data_reward=np.vstack([epi_mask_data_reward,epi_mask_data])
    elif run==4:
        epi_mask_data_reward=np.vstack([epi_mask_data_reward,epi_mask_data])
    elif run==6:
        epi_mask_data_familiarization=epi_mask_data
    elif run==7:
        epi_mask_data_familiarization=np.vstack([epi_mask_data_familiarization,epi_mask_data])
    elif run==8:
        epi_mask_data_decision=epi_mask_data
    elif run==9:
        epi_mask_data_decision=np.vstack([epi_mask_data_decision,epi_mask_data])
    else:
        epi_mask_data_postfaces=epi_mask_data
    
    affine_mat = avg_mask.affine #should be the same as the epi data
    avg_mask.shape
    coords = np.where(avg_mask.get_fdata())
    bold_vol=[]
    bold_vol=np.zeros((avg_mask.shape[0], avg_mask.shape[1], avg_mask.shape[2], epi_mask_data.shape[0]))
    bold_vol[coords[0], coords[1], coords[2], :] = epi_mask_data.T
    print('epi_mask_data shape:', bold_vol.shape)
    print('')
    
    if run==5:
        real_run=1
        output_name = (out_dir + 'ses-02/' + '%s_ses-02_task-postfaces_run-0%i_space-T1w_desc-preproc_bold_trim%dand%dTRs_normalized.nii.gz' % (sub, real_run, n_trunc_beginning, n_trunc_end))
    elif run==6:
        real_run=1
        output_name = (out_dir + 'ses-02/' + '%s_ses-02_task-familiarization_run-0%i_space-T1w_desc-preproc_bold_trim%dand%dTRs_normalized.nii.gz' % (sub, real_run, n_trunc_beginning, n_trunc_end))
    elif run==7:
        real_run=2
        output_name = (out_dir + 'ses-02/' + '%s_ses-02_task-familiarization_run-0%i_space-T1w_desc-preproc_bold_trim%dand%dTRs_normalized.nii.gz' % (sub, real_run, n_trunc_beginning, n_trunc_end))
    elif run==8:
        real_run=1
        output_name = (out_dir + 'ses-02/' + '%s_ses-02_task-decision_run-0%i_space-T1w_desc-preproc_bold_trim%dand%dTRs_normalized.nii.gz' % (sub, real_run, n_trunc_beginning, n_trunc_end))
    elif run==9:
        real_run=2
        output_name = (out_dir + 'ses-02/' + '%s_ses-02_task-decision_run-0%i_space-T1w_desc-preproc_bold_trim%dand%dTRs_normalized.nii.gz' % (sub, real_run, n_trunc_beginning, n_trunc_end))
    else:
        output_name = (out_dir + 'ses-02/' + '%s_ses-02_task-reward_run-0%i_space-T1w_desc-preproc_bold_trim%dand%dTRs_normalized.nii.gz' % (sub, run, n_trunc_beginning, n_trunc_end))
    
    # Save the truncated, normalized data
    print('Saving trimmed and normalized volume for run',run, output_name)
    print('output filename:', output_name)
    bold_nii = nib.Nifti1Image(bold_vol, affine_mat) #convert to nifti
    hdr = bold_nii.header  #get a handle for the .nii file's header
    hdr.set_zooms((orig_dimsize[0], orig_dimsize[1], orig_dimsize[2], orig_dimsize[3]))
    nib.save(bold_nii, output_name)
        
print('Volumes saved')


## Transform and save concatenated data

In [ ]:
avg_mask.shape
coords = np.where(avg_mask.get_fdata())
dimsize=avg_mask.header.get_zooms()
print('Voxel dimensions:', dimsize)

affine_mat = avg_mask.affine  # What is the orientation of the data

# transform concatenated volumes and save

# FAMILIARIZATION
output_name = out_dir + 'ses-02/' + '%s_ses-02_task-familiarization_run-ALL_space-T1w_desc-preproc_bold_trim%dand%dTRs_normalized.nii.gz' % (sub, n_trunc_beginning, n_trunc_end)
bold_vol=[]
bold_vol =np.zeros((avg_mask.shape[0], avg_mask.shape[1], avg_mask.shape[2], epi_mask_data_familiarization.shape[0]))
bold_vol[coords[0], coords[1], coords[2], :] = epi_mask_data_familiarization.T

print('concatenate familiarization runs')
print('avg_mask shape:', avg_mask.shape)
print('epi_mask_data shape:', bold_vol.shape)
print('epi_mask_data_familiarization shape(timepoints, voxels):', epi_mask_data_familiarization.shape)
print('')
print('output filename:', output_name)
print('')
bold_nii = nib.Nifti1Image(bold_vol, affine_mat)
hdr = bold_nii.header  # get a handle for the .nii file's header
print('Dimensions:', orig_dimsize) #4th dimension is TR
print('')
hdr.set_zooms((orig_dimsize[0], orig_dimsize[1], orig_dimsize[2], orig_dimsize[3]))
nib.save(bold_nii, output_name)
print('Volume saved')

# REWARD
output_name = out_dir + 'ses-02/' + '%s_ses-02_task-reward_run-ALL_space-T1w_desc-preproc_bold_trim%dand%dTRs_normalized.nii.gz' % (sub, n_trunc_beginning, n_trunc_end)
bold_vol=[]
bold_vol =np.zeros((avg_mask.shape[0], avg_mask.shape[1], avg_mask.shape[2], epi_mask_data_reward.shape[0]))
bold_vol[coords[0], coords[1], coords[2], :] = epi_mask_data_reward.T

print('concatenate reward runs')
print('avg_mask shape:', avg_mask.shape)
print('epi_mask_data shape:', bold_vol.shape)
print('epi_mask_data_reward shape(timepoints, voxels):', epi_mask_data_reward.shape)
print('')
print('output filename:', output_name)
print('')
bold_nii = nib.Nifti1Image(bold_vol, affine_mat)
hdr = bold_nii.header  # get a handle for the .nii file's header
print('Dimensions:', orig_dimsize) #4th dimension is TR
print('')
hdr.set_zooms((orig_dimsize[0], orig_dimsize[1], orig_dimsize[2], orig_dimsize[3]))
nib.save(bold_nii, output_name)
print('Volume saved')

# DECISION
output_name = out_dir + 'ses-02/' + '%s_ses-02_task-decision_run-ALL_space-T1w_desc-preproc_bold_trim%dand%dTRs_normalized.nii.gz' % (sub, n_trunc_beginning, n_trunc_end)
bold_vol=[]
bold_vol =np.zeros((avg_mask.shape[0], avg_mask.shape[1], avg_mask.shape[2], epi_mask_data_decision.shape[0]))
bold_vol[coords[0], coords[1], coords[2], :] = epi_mask_data_decision.T

print('concatenate decision runs')
print('avg_mask shape:', avg_mask.shape)
print('epi_mask_data shape:', bold_vol.shape)
print('epi_mask_data_decision shape(timepoints, voxels):', epi_mask_data_decision.shape)
print('')
print('output filename:', output_name)
print('')
bold_nii = nib.Nifti1Image(bold_vol, affine_mat)
hdr = bold_nii.header  # get a handle for the .nii file's header
print('Dimensions:', orig_dimsize) #4th dimension is TR
print('')
hdr.set_zooms((orig_dimsize[0], orig_dimsize[1], orig_dimsize[2], orig_dimsize[3]))
nib.save(bold_nii, output_name)
print('Volume saved')

## Plot a volume from each run to check normalization step

In [ ]:
# vol_num=0
# for run in range(1,svd_n_runs+1):
#     func_name = (out_dir + '%s_ses-00_task-localizer_run-0%i_space-T1w_desc-preproc_bold_trim%d_norm.nii.gz' % (sub, run, n_trunc))
#     image.load_img(func_name)
#     first_vol = image.index_img(func_name,vol_num)
#     #mean_func = mean_img(func_name)
#     plot_epi(first_vol)

### 3.1. Plot a voxel time-series <a id="plot_voxel"></a>

After masking, the fMRI dataset at this stage (found in the variable epi_mask_data that was created in the cell above) is in the format rows=time (i.e. 310 rows referring to 310 TRs) and columns=voxels (i.e. the number of voxels in the mask).

In [ ]:
# Plot value of voxel_id through time point 650 (there are only 618 time points)
voxel_id = 200

f, ax = plt.subplots(1,1, figsize=(14,5))
ax.plot(epi_mask_data_postfaces[:, voxel_id])
ax.set_title('Voxel time series, POSTFACES, voxel id = %d' % voxel_id)
ax.set_xlabel('TR')
ax.set_ylabel('Voxel Intensity')

f, ax = plt.subplots(1,1, figsize=(14,5))
ax.plot(epi_mask_data_familiarization[:, voxel_id])
ax.set_title('Voxel time series, FAMILIARIZATION, voxel id = %d' % voxel_id)
ax.set_xlabel('TR')
ax.set_ylabel('Voxel Intensity')

f, ax = plt.subplots(1,1, figsize=(14,5))
ax.plot(epi_mask_data_reward[:, voxel_id])
ax.set_title('Voxel time series, REWARD, voxel id = %d' % voxel_id)
ax.set_xlabel('TR')
ax.set_ylabel('Voxel Intensity')

f, ax = plt.subplots(1,1, figsize=(14,5))
ax.plot(epi_mask_data_decision[:, voxel_id])
ax.set_title('Voxel time series, DECISION, voxel id = %d' % voxel_id)
ax.set_xlabel('TR')
ax.set_ylabel('Voxel Intensity')

# plt.figure(figsize=(10,10))
# plt.imshow(epi_mask_data_all[200:300,voxel_id-100:voxel_id+100])
# plt.colorbar()
# plt.show()

## Check mean and standard deviation of normalized data (reward phase only) 

In [ ]:
x_mean = np.mean(epi_mask_data_reward, axis=0)
x_std = np.std(epi_mask_data_reward, axis=0, dtype=np.float64)

print('the mean of 1st few time points:\n', x_mean[0:10])
print('')
print('the std of 1st few time points:\n', x_std[0:10])
print('')
print(np.shape(x_mean))
print(np.shape(x_std))
print('')
print(np.amin(x_mean), np.amax(x_mean))
print(np.amin(x_std), np.amax(x_std))

# print(x_std)

In [ ]:
f, axes = plt.subplots(1, 2, figsize = (14,4))

n_bins = 20

axes[0].hist(x_mean, bins = n_bins)
axes[0].set_title('distribution of means')
axes[0].set_xlabel('mean values')
axes[0].set_ylabel('counts')
axes[0].xaxis.set_major_locator(plt.MaxNLocator(3))

axes[1].hist(x_std, bins = n_bins)
axes[1].set_title('distribution of stds')
axes[1].set_xlabel('std values')
axes[1].set_ylabel('counts')
axes[1].xaxis.set_major_locator(plt.MaxNLocator(3))
axes[1].get_xaxis().get_major_formatter().set_useOffset(False)

plt.show()